In [2]:
import os
import pandas as pd
import subprocess
import shutil
import time
from utils.helper import extract_metrics_from_output, convert_results_into_df, running_time

In [4]:
cuda_device = "2"

# Dynamic variables
pred_lens = [24, 96, 168]
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
num_cols = [5, 5, 3, 3, 3]

os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

In [5]:
log_dir = f"logs/timellm/"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Dynamic variables
seq_len = 512
model = "TimeLLM"

# Log file with all the results in 1 file
log_file_path = f"{log_dir}/{model}.log"

# Parameters for tuning,but default
lr = 0.01
train_epochs = 100
#n_heads = 16
#e_layers = 3
#d_model = 128
#d_ff = 256
d_model=32
d_ff=128
#dropout = 0.2
#patch_len = 32
#stride = 16
#batch_size = 128 
batch_size = 24

# List to store the results
timellm_results = []

In [8]:
start = time.time()

# Open log file
with open(log_file_path, "w") as log_file:
    
    for i, country in enumerate(countries):
        country_start = time.time()
        statement_1 = f"\n=== Starting experiments for country: {country} ===\n"
        log_file.write(statement_1)
        print(statement_1)

        for pred_len in pred_lens:
            pred_len_start = time.time()
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2)

            # Command to run script with parameters
            command = f"""
            python -m accelerate.commands.launch --mixed_precision bf16 --num_processes=1 --num_machines 1 --dynamo_backend "no" --main_process_port "01025" ./Time-LLM/run_main.py \
              --task_name long_term_forecast \
              --is_training 1 \
              --root_path ./datasets/ \
              --data_path {country}_data.csv \
              --model_id {i+1} \
              --model {model} \
              --data {country} \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --factor 3 \
              --enc_in {num_cols[i]} \
              --c_out {num_cols[i]} \
              --des 'Exp' \
              --itr 1 \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --batch_size {batch_size} \
              --learning_rate {lr} \
              --llm_model "GPT2" \
              --llm_dim 768 \
              --llm_layers 12 \
              --train_epochs {train_epochs} \
              --model_comment {model}+{country}
            """

            # Run command and log output
            log_file.write(f"\n--- Running model for {country}, pred_len={pred_len} ---\n")
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture and log output in real-time
            output = []
            for line in process.stdout:
                if verbose:  # Only print if verbose is True
                    print(line, end='')
                    output.append(line)
                
                print(line, end='')
                log_file.write(line)

            process.wait()  # Wait for process to finish

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr=1)
            timellm_results.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': iteration_metrics[0],
                'RMSE': iteration_metrics[1],
                'MAE': iteration_metrics[2]
                })

            # Time tracking for pred_len
            pred_len_end = time.time()
            hours_int, mins_int, secs_int = running_time(pred_len_start, pred_len_end)
            statement_3 = f"Intermediate time for {country} and pred_len {pred_len}: {hours_int:0>2}h:{mins_int:0>2}m:{secs_int:05.2f}s\n"
            log_file.write(statement_3)
            print(statement_3)

        # Time tracking for each country
        country_end = time.time()
        hours_c, mins_c, secs_c = running_time(country_start, country_end)
        statement_4 = f"Intermediate time for {country}: {hours_c:0>2}h:{mins_c:0>2}m:{secs_c:05.2f}s\n"
        log_file.write(statement_4)
        print(statement_4)

    # Total time
    end = time.time()
    hours, mins, secs = running_time(start, end)
    statement_5 = f"Total time: {hours:0>2}h:{mins:0>2}m:{secs:05.2f}s\n"
    log_file.write(statement_5)
    print(statement_5)



=== Starting experiments for country: DE ===


=== Starting experiments for pred_len: 24 ===

train 143005
val 31085
test 31085
Hourly data detailing load (electricity consumption), solar generation, and wind generation. These metrics are crucial in the electric power demand planning. 
[2024-10-27 01:17:46,584] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-10-27 01:17:51,468] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.14.0, git-hash=unknown, git-branch=unknown
[2024-10-27 01:17:51,468] [INFO] [comm.py:637:init_distributed] cdb=None
[2024-10-27 01:17:51,468] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2024-10-27 01:17:51,629] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=141.20.21.42, master_port=29500
[2024-10-27 01:17:51,629] [INFO] [comm.py:668:init_distribute

KeyboardInterrupt: 